In [1]:
from natasha import (
    NamesExtractor,
    DatesExtractor,
    AddrExtractor,
    MorphVocab,
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc
)


file_path = 'article.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    article_text = file.read()

#////////////////////////////////////////////////////////////////////////

# Initialize Natasha NER components
# Инициализировать компоненты Natasha NER

morph_vocab = MorphVocab()
# handling morphological information in the text and breaking words down into the smallest grammatical units and analyzing their grammatical properties
# обработка морфологической информации в тексте, разбиение слов на мельчайшие грамматические единицы и анализ их грамматических свойств
segmenter = Segmenter()
# segmenting text into tokens or words
# сегментирование текста на лексемы или слова
emb = NewsEmbedding()
# pre-trained word embedding model specifically trained on news articles. Word embeddings are dense vector representations of words in a high-dimensional space, where similar words are represented by similar vectors
# предварительно обученная модель встраивания слов, специально обученная на новостных статьях. Встраивания слов - это плотные векторные представления слов в многомерном пространстве, где похожие слова представлены похожими векторами
morph_tagger = NewsMorphTagger(emb)
# assigns morphological tags to each word token, indicating its grammatical properties such as part of speech, tense, case, gender, and number
# присваивает морфологические теги каждой лексеме слова, указывая на ее грамматические свойства, такие как часть речи, время, падеж, род и число.
syntax_parser = NewsSyntaxParser(emb)
# analyzes the relationships between words in a sentence
# анализирует отношения между словами в предложении
ner_tagger = NewsNERTagger(emb)
# identifies and classifies named entities such as persons, organizations, locations, dates, etc., based on their context and surrounding words
# идентифицирует и классифицирует именованные объекты, такие как лица, организации, местоположения, даты и т.д., на основе их контекста и окружающих слов

#////////////////////////////////////////////////////////////////////////

# Initialize extractors for different types of named entities
# Инициализировать экстракторы для различных типов именованных объектов

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)



#//////////////////////////////////////////////////////////////////////

# Apply it to the article text
# Применить к тексту статьи

doc = Doc(article_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
doc.tag_ner(ner_tagger)


# Extract named entities from the Text
# Извлеките именованные объекты из текста
named_entities = doc.spans
for span in named_entities:
    print(span.text, span.start, span.stop, span.type)



Фёдор Михайлович Достоевский 0 28 PER
Москве 51 57 LOC
Петербург 128 137 LOC
Главное инженерное училище 154 180 ORG
Достоевский 192 203 PER
Петербургскую инженерную команду 260 292 ORG
Достоевского 382 394 PER
Белинского 428 438 LOC
Белинским 604 613 PER
Петрашевского 641 654 PER
Достоевский 868 879 PER
Петрашевского 911 924 PER
Семёновском плацу петрашевцам 967 996 ORG
Тобольске 1154 1163 LOC
Достоевского 1191 1203 PER
Евангелие 1305 1314 PER
Достоевского 1501 1513 PER
Христос 1686 1693 PER
Достоевский 1847 1858 PER
Марией Исаевой 1872 1886 PER
А.И. Исаева 1905 1916 PER
Петербург 1965 1974 LOC
Михаилом 1996 2004 PER
Время 2028 2033 ORG
Достоевского 2263 2275 PER
Эпоха 2429 2434 ORG
Анне Сниткиной 2542 2556 PER
Достоевский 2692 2703 PER
Старая Русса 2774 2786 LOC
Новгородской губернии 2787 2808 LOC
Ф.М. Достоевский 3188 3204 PER
Похоронен 3216 3225 PER
Александро-Невской лавре 3237 3261 ORG
Санкт-Петербурге 3264 3280 LOC


In [2]:
# Print only the names
filtered_entities = []

for span in named_entities:
    if span.type == 'PER':
        text = span.text
        filtered_text = ' '.join(word for word in text.split() if not word.isdigit())
        filtered_span = doc.text[span.start:span.stop].replace(text, filtered_text)
        filtered_entities.append(filtered_span)

print("Отфильтрованные объекты:\n")
print("-----------------------\n")
for span in filtered_entities:
    print(span)



Отфильтрованные объекты:

-----------------------

Фёдор Михайлович Достоевский
Достоевский
Достоевского
Белинским
Петрашевского
Достоевский
Петрашевского
Достоевского
Евангелие
Достоевского
Христос
Достоевский
Марией Исаевой
А.И. Исаева
Михаилом
Достоевского
Анне Сниткиной
Достоевский
Ф.М. Достоевский
Похоронен


In [3]:
named_entities = doc.spans

# Rule 1 - Print persons with associated locations
# Правило 1 - Печать лиц с указанием местоположения
print("Персоны, занимающие соответствующие должности:\n")
print("----------------------------------------\n")
for span in named_entities:
    if span.type == 'LOC':
        location_span = span
        person_span = None
        for neighbor_span in named_entities:
            if neighbor_span.start < location_span.start and neighbor_span.type == 'PER':
                person_span = neighbor_span
                break
        if person_span:
            print("Личность:", person_span.text, "| Расположение:", location_span.text)


Персоны, занимающие соответствующие должности:

----------------------------------------

Личность: Фёдор Михайлович Достоевский | Расположение: Москве
Личность: Фёдор Михайлович Достоевский | Расположение: Петербург
Личность: Фёдор Михайлович Достоевский | Расположение: Белинского
Личность: Фёдор Михайлович Достоевский | Расположение: Тобольске
Личность: Фёдор Михайлович Достоевский | Расположение: Петербург
Личность: Фёдор Михайлович Достоевский | Расположение: Старая Русса
Личность: Фёдор Михайлович Достоевский | Расположение: Новгородской губернии
Личность: Фёдор Михайлович Достоевский | Расположение: Санкт-Петербурге


In [4]:
# Rule 2: Print all named entities with their types
# Правило 2: Выведите все именованные сущности с их типами
print("Все именованные сущности и типы:\n")
print("--------------------------------\n")
for span in named_entities:
    print(span.text, span.type)




Все именованные сущности и типы:

--------------------------------

Фёдор Михайлович Достоевский PER
Москве LOC
Петербург LOC
Главное инженерное училище ORG
Достоевский PER
Петербургскую инженерную команду ORG
Достоевского PER
Белинского LOC
Белинским PER
Петрашевского PER
Достоевский PER
Петрашевского PER
Семёновском плацу петрашевцам ORG
Тобольске LOC
Достоевского PER
Евангелие PER
Достоевского PER
Христос PER
Достоевский PER
Марией Исаевой PER
А.И. Исаева PER
Петербург LOC
Михаилом PER
Время ORG
Достоевского PER
Эпоха ORG
Анне Сниткиной PER
Достоевский PER
Старая Русса LOC
Новгородской губернии LOC
Ф.М. Достоевский PER
Похоронен PER
Александро-Невской лавре ORG
Санкт-Петербурге LOC


In [5]:
# Rule 3 - Print All named entities and dates 
# Правило 3 - Вывести все названные сущности и даты 
named_entities = doc.spans
dates = [match.fact.year for match in dates_extractor(article_text)]

filtered_entities_with_years = []
for span in named_entities:
    text = span.text.split()[0]
    for date in dates:
        if text in article_text and str(date) in article_text:
            filtered_entities_with_years.append((text, date))

print("Все названные сущности с годами:\n")
print("--------------------------------\n")
for entity, year in filtered_entities_with_years:
    print(f"{entity}: {year}")


Все названные сущности с годами:

--------------------------------

Фёдор: 1837
Фёдор: 1842
Фёдор: 1845
Фёдор: 1846
Фёдор: 1847
Фёдор: 1849
Фёдор: 1857
Фёдор: 1859
Фёдор: 1860
Фёдор: 1861
Фёдор: 1864
Фёдор: 1866
Фёдор: 1868
Фёдор: 1881
Москве: 1837
Москве: 1842
Москве: 1845
Москве: 1846
Москве: 1847
Москве: 1849
Москве: 1857
Москве: 1859
Москве: 1860
Москве: 1861
Москве: 1864
Москве: 1866
Москве: 1868
Москве: 1881
Петербург: 1837
Петербург: 1842
Петербург: 1845
Петербург: 1846
Петербург: 1847
Петербург: 1849
Петербург: 1857
Петербург: 1859
Петербург: 1860
Петербург: 1861
Петербург: 1864
Петербург: 1866
Петербург: 1868
Петербург: 1881
Главное: 1837
Главное: 1842
Главное: 1845
Главное: 1846
Главное: 1847
Главное: 1849
Главное: 1857
Главное: 1859
Главное: 1860
Главное: 1861
Главное: 1864
Главное: 1866
Главное: 1868
Главное: 1881
Достоевский: 1837
Достоевский: 1842
Достоевский: 1845
Достоевский: 1846
Достоевский: 1847
Достоевский: 1849
Достоевский: 1857
Достоевский: 1859
Достоевский: 1860


Заметки:

Наташа эффективно идентифицировала и извлекала из текста именованные сущности, такие как люди, места и даты.